## Imported Libraries Overview

This cell imports essential Python libraries for geospatial analysis, data visualization, and statistical modeling:

- **Data Handling & Computation**: `numpy` (`np`), `pandas` (`pd`), `collections.defaultdict`
- **Geospatial Analysis**: `geopandas` (`gpd`), `shapely.geometry` (`Polygon`, `Point`), `contextily` (`ctx`), `folium`
- **Visualization**: `matplotlib.pyplot` (`plt`), `seaborn` (`sns`), `matplotlib.colors` (`Normalize`, `to_hex`)
- **Statistical Analysis**: `scipy.stats`, `scipy.optimize.curve_fit`, `kstest`
- **Date & Time**: `datetime.datetime`

In [2]:
from collections import defaultdict
import contextily as ctx
from datetime import datetime
import folium
import geopandas as gpd
from matplotlib.colors import Normalize, to_hex
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.optimize import curve_fit
from scipy.stats import kstest
from shapely.geometry import Polygon, Point
import seaborn as sns


## Data Cleaning and Preprocessing

Loads and process multi-sheet Excel data

1. **File Loading**: Reads all sheets from `2022_vitals.xlsx` without headers.
2. **Column Naming**: Assigns predefined column names for consistency.
3. **Data Alignment**: 
   - Fixes misaligned rows by detecting valid `deviceID`.
   - Ensures all rows have the correct number of columns.
4. **Filtering**:
   - Removes invalid or duplicate header rows.
   - Drops rows with zero values for latitude (`Lat`) and longitude (`Log`).
5. **Indexing**: Resets the index and assigns a sequential 1-based index.
6. **Output**: Saves the cleaned data to `2022_vitals_cleaned.xlsx` and previews it.

In [ ]:
# File path
file_path = "/Users/mayar/Desktop/MIT/Research_Fellow/ENERGY_SENSING/DATA/2022_vitals.xlsx"

# Specify the column names explicitly
column_names = [
    "deviceID", "Timestamp", "Lat", "Log", "SOC_batt", "temp_batt", "volatge_batt",
    "voltage_particle", "current_batt", "isCharging", "isCharginS", "isCharged",
    "Temp_int", "Hum_int", "solar_current", "Cellular_signal_strength","index"
]

# Load all sheets into a dictionary
sheets_dict = pd.read_excel(file_path, sheet_name=None, header=None)  # No header initially

# Process each sheet
processed_sheets = []
for sheet_name, sheet_data in sheets_dict.items():
    # Ensure the number of columns matches the expected number
    sheet_data = sheet_data.iloc[:, :len(column_names)]

    # Fix misaligned rows where the first column is invalid
    def fix_alignment(row):
        # Convert the row to a list
        row_list = row.tolist()

        # Find the first valid `deviceID` (assumes valid `deviceID` has > 5 characters)
        for i, value in enumerate(row_list):
            if isinstance(value, str) and len(value) > 5:  # Valid `deviceID` found
                # Align the row starting from the valid `deviceID`
                aligned_row = row_list[i:i + len(column_names)]
                # Ensure the row is padded or trimmed to match `column_names`
                return aligned_row + [None] * (len(column_names) - len(aligned_row))

        # If no valid `deviceID` is found, return a row of NaN
        return [None] * len(column_names)

    # Apply alignment fix to all rows
    sheet_data = sheet_data.apply(fix_alignment, axis=1, result_type="expand")
    
    # Assign column names
    sheet_data.columns = column_names

    # Drop rows where 'deviceID' is still invalid or starts with "deviceID"
    sheet_data = sheet_data[sheet_data['deviceID'].notna()]
    sheet_data = sheet_data[sheet_data['deviceID'] != "deviceID"]  # Remove rows starting with "deviceID"

    # Append processed sheet
    processed_sheets.append(sheet_data)

# Concatenate all sheets into one DataFrame
df = pd.concat(processed_sheets, ignore_index=True)

# Drop rows where Lat or Log is 0
df = df[(df['Lat'] != 0) & (df['Log'] != 0)]

# Correct the indexing column to start at 1 and increment sequentially
df.reset_index(drop=True, inplace=True)  # Reset pandas index
df['index'] = df.index + 1  # Create a 1-based index

# Save the cleaned data back to Excel (optional)
output_path = "/Users/mayar/Desktop/MIT/Research_Fellow/ENERGY_SENSING/DATA/2022_vitals_cleaned.xlsx"
df.to_excel(output_path, index=False)

# Print the cleaned data preview
print(df.head())


# **Spatiotemporal Binning and Stationary Period Detection**

This enables **spatial binning**, **stationary period detection**, and **temporal filtering** for robust movement analysis.

## **1. Timestamp Conversion**
The Unix timestamp $ T_i $ is converted into a standard datetime format:

$$
T_i^{\text{datetime}} = T_i^{\text{unix}} \times \frac{1}{86400} + \text{epoch}
$$

where:
- $ T_i^{\text{unix}} $ is the raw Unix timestamp in **seconds**,
- $ 86400 $ seconds = **1 day**,
- **Epoch** is the reference starting time (January 1, 1970).

## **2. Ensuring Numeric Latitude and Longitude**
We enforce that latitude ($ \text{Lat} $) and longitude ($ \text{Log} $) are real-valued:

$$
\text{Lat}, \text{Log} \in \mathbb{R}
$$

Non-numeric values are coerced to **NaN**.

## **3. Discretization into a 120m Grid**
### **3.1 Latitude Grid Resolution**
Since the **Earth's meridional circumference** is approximately **40,030 km**, the degree-to-meter conversion near the equator is:

$$
1^\circ \approx 111,320 \text{ meters}
$$

Thus, the spatial resolution of a **120m grid** in latitude is:

$$
\Delta \text{Lat} = \frac{120}{111320}
$$

The **grid-aligned latitude** is computed as:

$$
\text{Lat\_Grid} = \left\lfloor \frac{\text{Lat}}{\Delta \text{Lat}} \right\rfloor \times \Delta \text{Lat}
$$

### **3.2 Longitude Grid Resolution**
Unlike latitude, **longitude spacing** varies with latitude due to Earth’s curvature. The **longitude degree-to-meter conversion** is:

$$
1^\circ \approx 111320 \times \cos(\text{Lat})
$$

Thus, the **longitude resolution** at a given latitude is:

$$
\Delta \text{Log} = \frac{120}{111320 \cos(\text{Lat})}
$$

The **grid-aligned longitude** is computed as:

$$
\text{Log\_Grid} = \left\lfloor \frac{\text{Log}}{\Delta \text{Log}} \right\rfloor \times \Delta \text{Log}
$$

## **4. Sorting by Time and Device**
To track movement **chronologically** for each vehicle, we sort:

$$
\text{Sort}(df, \text{by}=[\text{deviceID}, \text{Timestamp}])
$$

## **5. Identifying Stationary Periods**
For each vehicle, we determine if it remained in the same grid cell over consecutive timestamps:

$$
\text{Same\_Grid}_i =
\begin{cases} 
1, & (\text{Lat\_Grid}_i = \text{Lat\_Grid}_{i-1}) \land (\text{Log\_Grid}_i = \text{Log\_Grid}_{i-1}) \\
0, & \text{otherwise}
\end{cases}
$$

where:
- $ \text{Same\_Grid}_i = 1 $ means no movement occurred.
- $ \text{Same\_Grid}_i = 0 $ means movement occurred.

## **6. Computing Time Spent in a Grid Cell**
The time difference between consecutive records within the same grid is:

$$
\Delta t_i = T_i - T_{i-1}
$$

The **total duration** a vehicle spends within a specific grid cell before moving is:

$$
\text{Cumulative\_Time}_{i} = \sum_{k=1}^{i} \Delta t_k
$$

where:
- The summation continues **until movement occurs**.

## **7. Assigning a Group ID to Each Stationary Period**
A **unique group identifier** is assigned to each stationary period using a cumulative sum:

$$
\text{Group}_i =
\sum_{j=1}^{i} (1 - \text{Same\_Grid}_j)
$$

Each transition into a **new grid cell** increments the group ID.

## **8. Removing Prolonged Stationary Vehicles**
Vehicles remaining in the **same grid for over 3 hours** (10,800 seconds) are excluded:

$$
\text{Remove } i \text{ if } \text{Cumulative\_Time}_i \geq 10,800 \text{ sec}
$$

## **9. Cleanup**
All intermediate columns used for calculations are dropped to optimize storage.


In [ ]:
# Convert Unix timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')

# Ensure 'Lat' and 'Log' are numeric
df['Lat'] = pd.to_numeric(df['Lat'], errors='coerce')
df['Log'] = pd.to_numeric(df['Log'], errors='coerce')

# Spatial Resolution: 120m grid
grid=120
lat_resolution = grid / 111320  
df['Lat_Grid'] = (df['Lat'] // lat_resolution) * lat_resolution

# Longitude resolution depends on latitude
df['Lon_Resolution'] = grid / (111320 * np.cos(np.radians(df['Lat'])))
df['Log_Grid'] = (df['Log'] // df['Lon_Resolution']) * df['Lon_Resolution']

# Drop auxiliary column
df = df.drop(columns=['Lon_Resolution'])

# Step 1: Sort by deviceID and Timestamp
df = df.sort_values(by=['deviceID', 'Timestamp'])

# Step 2: Detect continuous stationary periods
df['Prev_Lat_Grid'] = df.groupby('deviceID')['Lat_Grid'].shift(1)
df['Prev_Log_Grid'] = df.groupby('deviceID')['Log_Grid'].shift(1)
df['Prev_Timestamp'] = df.groupby('deviceID')['Timestamp'].shift(1)

# Step 3: Identify whether the taxi has stayed in the same grid
df['Same_Grid'] = (df['Lat_Grid'] == df['Prev_Lat_Grid']) & (df['Log_Grid'] == df['Prev_Log_Grid'])

# Step 4: Compute time spent in the grid continuously
df['Time_Diff'] = (df['Timestamp'] - df['Prev_Timestamp']).dt.total_seconds()

# Step 5: Assign a group ID that resets when the taxi leaves a grid
df['Group'] = (~df['Same_Grid']).cumsum()

# Step 6: Compute total time spent in each visit to the grid
df['Cumulative_Time'] = df.groupby(['deviceID', 'Lat_Grid', 'Log_Grid', 'Group'])['Time_Diff'].cumsum()

# Step 7: Remove vehicles that stayed continuously in the same grid for more than 3hours (10800 sec)
df = df[~(df['Cumulative_Time'] >= 10800)]

# Drop helper columns
df = df.drop(columns=['Prev_Lat_Grid', 'Prev_Log_Grid', 'Prev_Timestamp', 'Same_Grid', 'Time_Diff', 'Group', 'Cumulative_Time'])


## Spatial Grid Aggregation

This script generates a **120m x 120m geospatial grid** and counts the number of data points within each grid cell:

1. **Dynamic Boundary Definition**: 
   - Extracts min/max latitude and longitude from the dataset.
2. **Grid Construction**:
   - Defines **120m resolution** for latitude and dynamically calculates longitude resolution.
   - Iterates over the spatial extent to generate **polygonal grid cells**.
3. **GeoDataFrame Creation**:
   - Converts the grid into a `GeoDataFrame` (`grid_gdf`).
   - Converts data points into a `GeoDataFrame` (`df_gdf`).
4. **Spatial Aggregation**:
   - Checks which points fall within each grid cell.
   - Increments the count of data points within corresponding grid polygons.

In [5]:
# Dynamically define the bounds from the DataFrame
min_lat = df['Lat'].min()
max_lat = df['Lat'].max()
min_lon = df['Log'].min()
max_lon = df['Log'].max()

# Define grid size (120x120 meters)
grid_size = 120
lat_resolution = grid_size / 111320  # Approximate latitude resolution
lon_resolution_at_lat = lambda lat: grid_size / (111320 * np.cos(np.radians(lat)))

# Generate grid of polygons
grid = []
lat = min_lat
while lat < max_lat:
    lon = min_lon
    while lon < max_lon:
        lon_res = lon_resolution_at_lat(lat)
        grid.append(Polygon([
            (lon, lat),
            (lon + lon_res, lat),
            (lon + lon_res, lat + lat_resolution),
            (lon, lat + lat_resolution)
        ]))
        lon += lon_res
    lat += lat_resolution

# Create an empty GeoDataFrame for the grid
grid_gdf = gpd.GeoDataFrame({'geometry': grid, 'Count': 0}, crs="EPSG:4326")

# Create a GeoDataFrame for the points in df
df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Log'], df['Lat']), crs="EPSG:4326")

# Increment the count for each grid square containing points
for index, point in df_gdf.iterrows():
    match = grid_gdf.contains(point.geometry)
    if match.any():
        grid_gdf.loc[match.idxmax(), 'Count'] += 1


## Heatmap Visualization of Measurement Density

Visualize the spatial distribution of measurements using a **120m x 120m grid heatmap**:

1. **Fractional Power Scaling**:
   - Applies **gamma correction (0.3)** to the count data for better visibility (can be modified for viz purposes).
   - Ensures variations in density are more distinguishable.

2. **Grid Heatmap Plotting**:
   - Colors the grid cells based on scaled count values using the **jet colormap**.
   - Adjusts transparency (`alpha=0.4`) for better layering.

3. **Basemap Integration**:
   - Overlays the heatmap on an **OpenStreetMap** basemap for context.

4. **Plot Aesthetics**:
   - Titles the plot **"Heatmap of Measurements Across Stockholm (120x120m Grid)"**.
   - Removes axis labels for cleaner visualization.

In [ ]:
# Apply Fractional Power Scaling to 'Count'
gamma = 0.3  # Adjust between 0.2 - 0.5 for better visibility
grid_gdf['Scaled_Count'] = (grid_gdf['Count'] + 1) ** gamma

# Plot the heatmap
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

# Plot the grid using the scaled count with a jet colormap
grid_gdf.plot(
    column='Scaled_Count',
    cmap='jet',
    legend=True,
    alpha=0.4,
    ax=ax,
)

# Add a basemap
ctx.add_basemap(ax, crs=grid_gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)

# Adjust plot aesthetics
ax.set_title('Heatmap of Measurements Across Stockholm (120x120m Grid)', fontsize=12)
ax.set_axis_off()

plt.show()

## Sampling Frequency Analysis in High-Density Grid Cells

This script analyzes the **sampling frequency** in the **top 10 most frequently sensed grid cells**:

1. **Identify High-Activity Areas**:
   - Selects the **top 10 grid cells** with the highest measurement counts.

2. **Filter Relevant Data**:
   - Extracts sensor readings from these **top 10 grid cells**.

3. **Compute Time Differences**:
   - Calculates the time intervals between consecutive measurements per `deviceID` within each grid cell.

4. **Estimate Sampling Frequency**:
   - Computes the **mean sampling interval** per grid cell.
   - Converts it to **sampling frequency (Hz)** as **1 / mean time difference**.

5. **Extract Top Sampling Frequencies**:
   - Selects the **10 highest sampling frequencies** for analysis.
   - Converts results into a structured DataFrame for better readability.


In [7]:
# Select the top 10 most frequently sensed grid cells
top_10_cells = grid_gdf.nlargest(10, 'Count')

# Filter the data for points within these top 10 grid cells
df_top_cells = df_gdf[df_gdf.geometry.apply(lambda point: any(top_10_cells.contains(point)))].copy()  # Make a copy

# Compute the time differences for each device in each grid cell
df_top_cells['Time_Diff'] = df_top_cells.groupby(['Lat_Grid', 'Log_Grid', 'deviceID'])['Timestamp'].diff().dt.total_seconds()

# Calculate the mean sampling frequency (1 / mean time difference) per grid cell
sampling_frequency = df_top_cells.groupby(['Lat_Grid', 'Log_Grid'])['Time_Diff'].mean().dropna().apply(lambda x: 1 / x)
top_10_frequencies = sampling_frequency.nlargest(10)

# Convert to DataFrame for better visualization
top_10_frequencies_df = top_10_frequencies.reset_index()
top_10_frequencies_df.columns = ['Lat_Grid', 'Log_Grid', 'Sampling_Frequency (Hz)']

## Battery Depletion

Identify the **number of unique days** where at least one device’s battery **(SOC_batt)** dropped below **50%**:

1. **Extract Date Information**:
   - Converts the timestamp to **date-only format**.

2. **Filter for Battery Depletion Events**:
   - Selects records where `SOC_batt` is below **50%** (can be changed).

3. **Count Unique Affected Days**:
   - Computes the number of distinct days where a depletion event occurred.

In [ ]:
# Assuming df is already loaded with the necessary data
# Identify unique days where at least one device's SOC_batt dropped below 50%
df['Date'] = df['Timestamp'].dt.date  # Extract the date
days_with_depletion = df[df['SOC_batt'] < 50]['Date'].nunique()

# Display the number of days with a battery drop below 50%
days_with_depletion


# **Markov-Based Spatial Transition Modeling for Vehicle Movement and Sensor Depletion Analysis**

This methodology enables **data-driven mobility prediction** and **battery depletion analysis** using **Markovian dynamics**.

## **1. Assigning Vehicles to Grid Cells Using Polygon Containment**
Given a dataset of GPS points (`Lat`, `Log`), we spatially bin the data into a **120m x 120m** grid using polygon containment:

- Convert each GPS point into a **GeoDataFrame** (`df_gdf`).
- Each point is assigned to its corresponding grid cell using the spatial containment function:

  $$
  G(i) = \arg\max_j \mathbb{1}(p_i \in P_j)
  $$

  where:
  - $ p_i $ is the point geometry of record $ i $,
  - $ P_j $ is the polygon geometry of grid cell $ j $,
  - $ \mathbb{1}(p_i \in P_j) $ is an indicator function that is **1** if $ p_i $ is inside $ P_j $, else **0**.

- Any points that **do not match** a grid cell are treated as **outliers** and removed.

## **2. Temporal Sorting for Transition Analysis**
To analyze transitions between grid cells, the dataset is **sorted** by:

$$
\text{Sort}(df, \text{by}=[\text{deviceID}, \text{Timestamp}])
$$

where:
- `deviceID` ensures sorting is done **per vehicle**,
- `Timestamp` orders data points **chronologically**.

## **3. Identifying Sensor Depletion Events**
A **battery depletion threshold** is defined as:

$$
SOC_{\text{batt}} < 50\%
$$

where **State of Charge (SOC)** is the battery’s remaining capacity. We create a binary indicator:

$$
D_i = 
\begin{cases} 
1, & SOC_{\text{batt}, i} < 50\% \\
0, & \text{otherwise}
\end{cases}
$$

- **Pre-Depletion Data**: $ D_i = 0 $ (battery level above threshold).
- **Post-Depletion Data**: $ D_i = 1 $ (battery level below threshold).

## **4. Constructing Grid Cell Transitions**
To model **spatial movement**, we define a transition as:

$$
T_i = (G_i, G_{i+1})
$$

where:
- $ G_i $ is the grid cell at time $ t_i $,
- $ G_{i+1} $ is the **next** grid cell at $ t_{i+1} $.

We compute the **state transitions** for each vehicle:

$$
\text{Next\_Grid\_Cell} = G_{i+1} = \text{shift}(G_i, -1)
$$

Dropping the last row for each vehicle ensures only **valid transitions** are included.

## **5. Constructing the Markov Transition Matrix**
A **first-order Markov model** is constructed, where each transition probability is estimated as:

$$
P(G_{i+1} | G_i) = \frac{N(G_i \to G_{i+1})}{\sum_{G_j} N(G_i \to G_j)}
$$

where:
- $ N(G_i \to G_{i+1}) $ is the **count** of observed transitions from $ G_i $ to $ G_{i+1} $,
- The denominator sums over **all possible** next states.

The **transition matrix** $ P $ is structured as:

$$
P = \begin{bmatrix}
P(G_1 | G_1) & P(G_2 | G_1) & \dots & P(G_n | G_1) \\
P(G_1 | G_2) & P(G_2 | G_2) & \dots & P(G_n | G_2) \\
\vdots & \vdots & \ddots & \vdots \\
P(G_1 | G_n) & P(G_2 | G_n) & \dots & P(G_n | G_n)
\end{bmatrix}
$$

This is a **stochastic matrix**, where each row sums to **1**:

$$
\sum_{G_{i+1}} P(G_{i+1} | G_i) = 1, \quad \forall G_i
$$

## **6. Predicting the Most Likely Next Grid Cell**
For a given grid cell $ G_i $, the predicted **next location** is:

$$
G_{\text{predicted}} = \arg\max_{G_{i+1}} P(G_{i+1} | G_i)
$$

This follows a **greedy decision rule**, selecting the most probable transition based on historical data.

## **7. Validation Against Post-Depletion Data**
For vehicles that **experienced battery depletion**:

- The predicted transition is compared to the **actual** next grid cell.
- A **correct prediction** is counted if:

  $$
  G_{\text{predicted}} = G_{\text{actual}}
  $$

- The **prediction accuracy** is computed as:

  $$
  \text{Accuracy} = \frac{\sum \mathbb{1}(G_{\text{predicted}} = G_{\text{actual}})}{N_{\text{post-depletion}}}
  $$

where $ N_{\text{post-depletion}} $ is the total number of post-depletion data points.

## **8. Output and Insights**
- The **transition matrix** is saved for further analysis.
- The **post-depletion validation results** are stored.
- The **top 10 most likely transitions** are displayed, providing insight into dominant movement patterns.

In [ ]:
## 1ST ORDER MC

# Step 1: Assign Vehicles to Grid Cells Using Polygon Containment
df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Log'], df['Lat']), crs="EPSG:4326")

# Function to find which grid cell a point belongs to
def find_grid_cell(point, grid_gdf):
    match = grid_gdf.contains(point.geometry)
    if match.any():
        return match.idxmax()  # Return index of matching grid cell
    else:
        return None  # No match found

# Apply function to assign each GPS point to a grid cell
df_gdf['Grid_Cell'] = df_gdf.apply(lambda row: find_grid_cell(row, grid_gdf), axis=1)

# Drop rows where no grid cell was matched (outliers)
df_gdf = df_gdf.dropna(subset=['Grid_Cell'])

# Step 2: Sort by deviceID and Timestamp for Transition Analysis
df_gdf = df_gdf.sort_values(by=['deviceID', 'Timestamp'])

# Step 3: Identify Sensor Depletion (SOC_batt < 50)
depletion_threshold = 50
df_gdf['Depleted'] = df_gdf['SOC_batt'] < depletion_threshold

# Step 4: Separate Pre- and Post-Depletion Data
df_pre_depletion = df_gdf[~df_gdf['Depleted']].copy()
df_post_depletion = df_gdf[df_gdf['Depleted']].copy()

# Step 5: Create Transitions (from one grid cell to the next)
df_pre_depletion['Next_Grid_Cell'] = df_pre_depletion.groupby('deviceID')['Grid_Cell'].shift(-1)

# Drop last row per vehicle (no next transition available)
df_transitions = df_pre_depletion.dropna(subset=['Next_Grid_Cell'])

# Step 6: Build the Markov Transition Matrix
transition_counts = (
    df_transitions.groupby(['Grid_Cell', 'Next_Grid_Cell'])
    .size()
    .unstack(fill_value=0)
)
transition_probabilities = transition_counts.div(transition_counts.sum(axis=1), axis=0)  # Normalize to probabilities

# Step 7: Define a Function to Predict the Next Grid Cell
def predict_next_grid(current_grid, transition_matrix):
    if current_grid in transition_matrix.index:
        return transition_matrix.loc[current_grid].idxmax()  # Most likely transition
    else:
        return None  # No transition data available

# Step 8: Validate Predictions Using Post-Depletion Data
df_post_depletion['Predicted_Grid_Cell'] = df_post_depletion['Grid_Cell'].apply(
    lambda x: predict_next_grid(x, transition_probabilities)
)

# Step 9: Measure Prediction Accuracy
df_post_depletion['Correct_Prediction'] = df_post_depletion['Predicted_Grid_Cell'] == df_post_depletion['Grid_Cell']
accuracy = df_post_depletion['Correct_Prediction'].mean()

print(f"Prediction Accuracy: {accuracy:.2%}")

# # Step 10: Save the Transition Matrix and Post-Depletion Validation
# output_path_transition_matrix = "/Users/mayar/Desktop/MIT/Research_Fellow/ENERGY_SENSING/DATA/Transition_Matrix.xlsx"
# transition_probabilities.to_excel(output_path_transition_matrix)

# output_path_post_depletion = "/Users/mayar/Desktop/MIT/Research_Fellow/ENERGY_SENSING/DATA/Post_Depletion_Validation.xlsx"
# df_post_depletion.to_excel(output_path_post_depletion, index=False)

# # Step 11: Analyze and Print the Top 10 Most Likely Transitions
# most_likely_transitions = (
#     transition_probabilities.stack()
#     .reset_index()
#     .rename(columns={0: 'Probability', 'level_0': 'From_Grid', 'level_1': 'To_Grid'})
#     .sort_values(by='Probability', ascending=False)
# )

# print("Top 10 Most Likely Transitions:")
# print(most_likely_transitions.head(10))

In [ ]:
# 2ND ORDER MC
# Step 1: Assign Vehicles to Grid Cells Using Polygon Containment
df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Log'], df['Lat']), crs="EPSG:4326")

# Function to find which grid cell a point belongs to
def find_grid_cell(point, grid_gdf):
    match = grid_gdf.contains(point.geometry)
    if match.any():
        return match.idxmax()  # Return index of matching grid cell
    else:
        return None  # No match found

# Apply function to assign each GPS point to a grid cell
df_gdf['Grid_Cell'] = df_gdf.apply(lambda row: find_grid_cell(row, grid_gdf), axis=1)

# Drop rows where no grid cell was matched (outliers)
df_gdf = df_gdf.dropna(subset=['Grid_Cell'])

# Step 2: Sort by deviceID and Timestamp for Transition Analysis
df_gdf = df_gdf.sort_values(by=['deviceID', 'Timestamp'])

# Step 3: Identify Sensor Depletion (SOC_batt < 50)
depletion_threshold = 50
df_gdf['Depleted'] = df_gdf['SOC_batt'] < depletion_threshold

# Step 4: Separate Pre- and Post-Depletion Data
df_pre_depletion = df_gdf[~df_gdf['Depleted']].copy()
df_post_depletion = df_gdf[df_gdf['Depleted']].copy()

# Step 5: Create Second-Order Transitions (considering two previous grid cells)
df_pre_depletion['Prev_Grid_Cell'] = df_pre_depletion.groupby('deviceID')['Grid_Cell'].shift(1)
df_pre_depletion['Next_Grid_Cell'] = df_pre_depletion.groupby('deviceID')['Grid_Cell'].shift(-1)

# Drop rows where not enough history is available
df_transitions_2nd = df_pre_depletion.dropna(subset=['Prev_Grid_Cell', 'Next_Grid_Cell'])

# Step 6: Build the Second-Order Transition Matrix
transition_counts_2nd = (
    df_transitions_2nd.groupby(['Prev_Grid_Cell', 'Grid_Cell', 'Next_Grid_Cell'])
    .size()
    .unstack(fill_value=0)
)
transition_probabilities_2nd = transition_counts_2nd.div(transition_counts_2nd.sum(axis=1), axis=0)  # Normalize to probabilities

# Step 7: Define Prediction Function for Second-Order Markov Chain
def predict_next_grid_2nd(prev_grid, current_grid, transition_matrix):
    key = (prev_grid, current_grid)
    if key in transition_matrix.index:
        return transition_matrix.loc[key].idxmax()  # Most likely transition
    else:
        return None  # No transition data available

# Step 8: Apply Second-Order Prediction to Post-Depletion Data
df_post_depletion['Prev_Grid_Cell'] = df_post_depletion.groupby('deviceID')['Grid_Cell'].shift(1)
df_post_depletion['Predicted_Grid_Cell_2nd'] = df_post_depletion.apply(
    lambda row: predict_next_grid_2nd(row['Prev_Grid_Cell'], row['Grid_Cell'], transition_probabilities_2nd), axis=1
)

# Step 9: Measure Prediction Accuracy for Second-Order Markov Chain
df_post_depletion['Correct_Prediction_2nd'] = df_post_depletion['Predicted_Grid_Cell_2nd'] == df_post_depletion['Grid_Cell']
accuracy_2nd_order = df_post_depletion['Correct_Prediction_2nd'].mean()

print(f"Second-Order Markov Chain Prediction Accuracy: {accuracy_2nd_order:.2%}")

# # Step 10: Save the Second-Order Transition Matrix and Post-Depletion Validation
# output_path_transition_matrix_2nd = "/Users/mayar/Desktop/MIT/Research_Fellow/ENERGY_SENSING/DATA/Transition_Matrix_2nd_Order.xlsx"
# transition_probabilities_2nd.to_excel(output_path_transition_matrix_2nd)

# output_path_post_depletion_2nd = "/Users/mayar/Desktop/MIT/Research_Fellow/ENERGY_SENSING/DATA/Post_Depletion_Validation_2nd.xlsx"
# df_post_depletion.to_excel(output_path_post_depletion_2nd, index=False)

# # Step 11: Analyze and Print the Top 10 Most Likely Transitions for the Second-Order Markov Chain
# most_likely_transitions_2nd = (
#     transition_probabilities_2nd.stack()
#     .reset_index()
#     .rename(columns={0: 'Probability', 'level_0': 'Prev_Grid', 'level_1': 'Current_Grid', 'level_2': 'To_Grid'})
#     .sort_values(by='Probability', ascending=False)
# )

# print("Top 10 Most Likely Transitions (Second-Order Markov Chain):")
# print(most_likely_transitions_2nd.head(10))


In [ ]:
## HYBRID 1ST & 2ND ORDER MC

# Step 1: Assign Vehicles to Grid Cells Using Polygon Containment
df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Log'], df['Lat']), crs="EPSG:4326")

# Function to find which grid cell a point belongs to
def find_grid_cell(point, grid_gdf):
    match = grid_gdf.contains(point.geometry)
    if match.any():
        return match.idxmax()  # Return index of matching grid cell
    else:
        return None  # No match found

# Apply function to assign each GPS point to a grid cell
df_gdf['Grid_Cell'] = df_gdf.apply(lambda row: find_grid_cell(row, grid_gdf), axis=1)

# Drop rows where no grid cell was matched (outliers)
df_gdf = df_gdf.dropna(subset=['Grid_Cell'])

# Step 2: Sort by deviceID and Timestamp for Transition Analysis
df_gdf = df_gdf.sort_values(by=['deviceID', 'Timestamp'])

# Step 3: Identify Sensor Depletion (SOC_batt < 50)
depletion_threshold = 50
df_gdf['Depleted'] = df_gdf['SOC_batt'] < depletion_threshold

# Step 4: Separate Pre- and Post-Depletion Data
df_pre_depletion = df_gdf[~df_gdf['Depleted']].copy()
df_post_depletion = df_gdf[df_gdf['Depleted']].copy()

# Step 5: Create First-Order Transitions
df_pre_depletion['Next_Grid_Cell'] = df_pre_depletion.groupby('deviceID')['Grid_Cell'].shift(-1)

# Create Second-Order Transitions (considering two previous grid cells)
df_pre_depletion['Prev_Grid_Cell'] = df_pre_depletion.groupby('deviceID')['Grid_Cell'].shift(1)

# Drop rows where not enough history is available
df_transitions_1st = df_pre_depletion.dropna(subset=['Next_Grid_Cell'])
df_transitions_2nd = df_pre_depletion.dropna(subset=['Prev_Grid_Cell', 'Next_Grid_Cell'])

# Step 6: Build Transition Matrices
## First-Order Markov Chain
transition_counts_1st = (
    df_transitions_1st.groupby(['Grid_Cell', 'Next_Grid_Cell'])
    .size()
    .unstack(fill_value=0)
)
transition_probabilities_1st = transition_counts_1st.div(transition_counts_1st.sum(axis=1), axis=0)

## Second-Order Markov Chain
transition_counts_2nd = (
    df_transitions_2nd.groupby(['Prev_Grid_Cell', 'Grid_Cell', 'Next_Grid_Cell'])
    .size()
    .unstack(fill_value=0)
)
transition_probabilities_2nd = transition_counts_2nd.div(transition_counts_2nd.sum(axis=1), axis=0)

# Step 7: Define Hybrid Prediction Function
def predict_next_grid_hybrid(prev_grid, current_grid, transition_matrix_1st, transition_matrix_2nd):
    key = (prev_grid, current_grid)
    
    # Use Second-Order if available
    if key in transition_matrix_2nd.index:
        return transition_matrix_2nd.loc[key].idxmax()  # Most likely transition
    
    # If Second-Order not available, use First-Order
    elif current_grid in transition_matrix_1st.index:
        return transition_matrix_1st.loc[current_grid].idxmax()
    
    # If neither available, return None
    else:
        return None

# Step 8: Apply Hybrid Prediction to Post-Depletion Data
df_post_depletion['Prev_Grid_Cell'] = df_post_depletion.groupby('deviceID')['Grid_Cell'].shift(1)
df_post_depletion['Predicted_Grid_Cell_Hybrid'] = df_post_depletion.apply(
    lambda row: predict_next_grid_hybrid(row['Prev_Grid_Cell'], row['Grid_Cell'], transition_probabilities_1st, transition_probabilities_2nd), axis=1
)

# Step 9: Measure Prediction Accuracy for Hybrid Model
df_post_depletion['Correct_Prediction_Hybrid'] = df_post_depletion['Predicted_Grid_Cell_Hybrid'] == df_post_depletion['Grid_Cell']
accuracy_hybrid = df_post_depletion['Correct_Prediction_Hybrid'].mean()

print(f"Hybrid Markov Chain Prediction Accuracy: {accuracy_hybrid:.2%}")

# # Step 10: Save the Hybrid Transition Matrices and Post-Depletion Validation
# output_path_transition_matrix_1st = "/Users/mayar/Desktop/MIT/Research_Fellow/ENERGY_SENSING/DATA/Transition_Matrix_1st_Order.xlsx"
# transition_probabilities_1st.to_excel(output_path_transition_matrix_1st)

# output_path_transition_matrix_2nd = "/Users/mayar/Desktop/MIT/Research_Fellow/ENERGY_SENSING/DATA/Transition_Matrix_2nd_Order.xlsx"
# transition_probabilities_2nd.to_excel(output_path_transition_matrix_2nd)

# output_path_post_depletion_hybrid = "/Users/mayar/Desktop/MIT/Research_Fellow/ENERGY_SENSING/DATA/Post_Depletion_Validation_Hybrid.xlsx"
# df_post_depletion.to_excel(output_path_post_depletion_hybrid, index=False)

# # Step 11: Analyze and Print the Top 10 Most Likely Transitions for Hybrid Markov Chain
# most_likely_transitions_hybrid = (
#     transition_probabilities_2nd.stack()
#     .reset_index()
#     .rename(columns={0: 'Probability', 'level_0': 'Prev_Grid', 'level_1': 'Current_Grid', 'level_2': 'To_Grid'})
#     .sort_values(by='Probability', ascending=False)
# )

# print("Top 10 Most Likely Transitions (Hybrid Markov Chain):")
# print(most_likely_transitions_hybrid.head(10))


In [ ]:
# TIME-BASED MC

# Step 1: Assign Vehicles to Grid Cells Using Polygon Containment
df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Log'], df['Lat']), crs="EPSG:4326")

# Function to find which grid cell a point belongs to
def find_grid_cell(point, grid_gdf):
    match = grid_gdf.contains(point.geometry)
    if match.any():
        return match.idxmax()  # Return index of matching grid cell
    else:
        return None  # No match found

# Apply function to assign each GPS point to a grid cell
df_gdf['Grid_Cell'] = df_gdf.apply(lambda row: find_grid_cell(row, grid_gdf), axis=1)

# Drop rows where no grid cell was matched (outliers)
df_gdf = df_gdf.dropna(subset=['Grid_Cell'])

# Step 2: Sort by deviceID and Timestamp for Transition Analysis
df_gdf = df_gdf.sort_values(by=['deviceID', 'Timestamp'])

# Step 3: Identify Sensor Depletion (SOC_batt < 50)
depletion_threshold = 50
df_gdf['Depleted'] = df_gdf['SOC_batt'] < depletion_threshold

# Step 4: Extract Time-of-Day Bins
df_gdf['Hour'] = df_gdf['Timestamp'].dt.hour
df_gdf['Time_Bin'] = pd.cut(
    df_gdf['Hour'],
    bins=[0, 6, 12, 18, 24],  # Defining the four time periods
    labels=['Night', 'Morning', 'Afternoon', 'Evening'],
    right=False,
    include_lowest=True
)

# Step 5: Separate Pre- and Post-Depletion Data
df_pre_depletion = df_gdf[~df_gdf['Depleted']].copy()
df_post_depletion = df_gdf[df_gdf['Depleted']].copy()

# Step 6: Create First-Order Transitions Within Each Time Bin
df_pre_depletion['Next_Grid_Cell'] = df_pre_depletion.groupby(['deviceID', 'Time_Bin'])['Grid_Cell'].shift(-1)

# Drop last row per vehicle per time bin (no next transition available)
df_transitions_time = df_pre_depletion.dropna(subset=['Next_Grid_Cell'])

# Step 7: Build Time-Based Transition Matrices
transition_matrices_time = {}

for time_bin in df_transitions_time['Time_Bin'].unique():
    df_time_bin = df_transitions_time[df_transitions_time['Time_Bin'] == time_bin]
    
    transition_counts_time = (
        df_time_bin.groupby(['Grid_Cell', 'Next_Grid_Cell'])
        .size()
        .unstack(fill_value=0)
    )
    
    transition_matrices_time[time_bin] = transition_counts_time.div(transition_counts_time.sum(axis=1), axis=0)

# Step 8: Define Time-Based Prediction Function
def predict_next_grid_time_based(current_grid, time_bin, transition_matrices):
    if time_bin in transition_matrices and current_grid in transition_matrices[time_bin].index:
        return transition_matrices[time_bin].loc[current_grid].idxmax()  # Most likely transition
    else:
        return None  # No transition data available

# Step 9: Apply Time-Based Prediction to Post-Depletion Data
df_post_depletion['Predicted_Grid_Cell_Time_Based'] = df_post_depletion.apply(
    lambda row: predict_next_grid_time_based(row['Grid_Cell'], row['Time_Bin'], transition_matrices_time), axis=1
)

# Step 10: Measure Prediction Accuracy for Time-Based Markov Chain
df_post_depletion['Correct_Prediction_Time_Based'] = df_post_depletion['Predicted_Grid_Cell_Time_Based'] == df_post_depletion['Grid_Cell']
accuracy_time_based = df_post_depletion['Correct_Prediction_Time_Based'].mean()

print(f"Time-Based Markov Chain Prediction Accuracy: {accuracy_time_based:.2%}")

# # Step 11: Save the Time-Based Transition Matrices and Post-Depletion Validation
# for time_bin, matrix in transition_matrices_time.items():
#     output_path = f"/Users/mayar/Desktop/MIT/Research_Fellow/ENERGY_SENSING/DATA/Transition_Matrix_{time_bin}.xlsx"
#     matrix.to_excel(output_path)

# output_path_post_depletion_time = "/Users/mayar/Desktop/MIT/Research_Fellow/ENERGY_SENSING/DATA/Post_Depletion_Validation_Time_Based.xlsx"
# df_post_depletion.to_excel(output_path_post_depletion_time, index=False)

# # Step 12: Analyze and Print the Top 10 Most Likely Transitions for Time-Based Markov Chain
# most_likely_transitions_time_based = {}

# for time_bin, matrix in transition_matrices_time.items():
#     most_likely_transitions_time_based[time_bin] = (
#         matrix.stack()
#         .reset_index()
#         .rename(columns={0: 'Probability', 'level_0': 'Current_Grid', 'level_1': 'To_Grid'})
#         .sort_values(by='Probability', ascending=False)
#     )
    
#     print(f"Top 10 Most Likely Transitions ({time_bin}):")
#     print(most_likely_transitions_time_based[time_bin].head(10))


In [ ]:
import random
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

# Step 1: Assign Vehicles to Grid Cells Using Polygon Containment
df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Log'], df['Lat']), crs="EPSG:4326")

# Function to find which grid cell a point belongs to
def find_grid_cell(point, grid_gdf):
    match = grid_gdf.contains(point.geometry)
    if match.any():
        return match.idxmax()
    else:
        return None  # No match found

# Apply function to assign each GPS point to a grid cell
df_gdf['Grid_Cell'] = df_gdf.apply(lambda row: find_grid_cell(row, grid_gdf), axis=1)

# Drop rows where no grid cell was matched (outliers)
df_gdf = df_gdf.dropna(subset=['Grid_Cell'])

# Step 2: Sort by deviceID and Timestamp for Transition Analysis
df_gdf = df_gdf.sort_values(by=['deviceID', 'Timestamp'])

# Step 3: Identify Sensor Depletion (SOC_batt < 50)
depletion_threshold = 50
df_gdf['Depleted'] = df_gdf['SOC_batt'] < depletion_threshold

# Step 4: Separate Pre- and Post-Depletion Data
df_pre_depletion = df_gdf[~df_gdf['Depleted']].copy()
df_post_depletion = df_gdf[df_gdf['Depleted']].copy()

# Step 5: Extract Transitions (State, Action, Next State)
df_pre_depletion['Next_Grid_Cell'] = df_pre_depletion.groupby('deviceID')['Grid_Cell'].shift(-1)
df_transitions = df_pre_depletion.dropna(subset=['Next_Grid_Cell'])

# Step 6: Define RL Training Function for Hyperparameter Optimization
def train_rl(params):
    alpha = params['alpha']
    gamma = params['gamma']
    epsilon = params['epsilon']
    epsilon_decay = params['epsilon_decay']
    training_iterations = int(params['training_iterations'])

    # Initialize Q-Table
    q_table = {}

    # RL Training with Dynamic Exploration and Replay
    for _ in range(training_iterations):
        for _, row in df_transitions.iterrows():
            state = row['Grid_Cell']
            action = row['Next_Grid_Cell']

            if state not in q_table:
                q_table[state] = {}

            if action not in q_table[state]:
                q_table[state][action] = 0

            # Log-based reward normalization
            transition_count = df_transitions[(df_transitions['Grid_Cell'] == state) & (df_transitions['Next_Grid_Cell'] == action)].shape[0]
            reward = np.log(transition_count + 1)

            # Q-learning update
            max_future_q = max(q_table[state].values()) if q_table[state] else 0
            q_table[state][action] = (1 - alpha) * q_table[state][action] + alpha * (reward + gamma * max_future_q)

    # RL Prediction Function with Decaying Exploration
    def predict_next_grid_rl(state, q_table, epsilon):
        if state in q_table:
            if random.uniform(0, 1) < epsilon:
                return random.choice(list(q_table[state].keys()))  # Explore
            else:
                return max(q_table[state], key=q_table[state].get)  # Exploit
        return None

    # Apply RL-Based Prediction to Post-Depletion Data
    df_post_depletion['Predicted_Grid_Cell_RL'] = df_post_depletion['Grid_Cell'].apply(lambda x: predict_next_grid_rl(x, q_table, epsilon))

    # Reduce Epsilon Over Time
    epsilon = max(0.05, epsilon * epsilon_decay)

    # Calculate Accuracy
    df_post_depletion['Correct_Prediction_RL'] = df_post_depletion['Predicted_Grid_Cell_RL'] == df_post_depletion['Grid_Cell']
    accuracy = df_post_depletion['Correct_Prediction_RL'].mean()

    print(f"Trial Accuracy: {accuracy:.2%} | Params: {params}")

    # Stop if accuracy reaches 85%
    if accuracy >= 0.85:
        print("Target accuracy reached! Stopping optimization.")
        return {'loss': -accuracy, 'status': STATUS_OK, 'params': params}

    return {'loss': -accuracy, 'status': STATUS_OK}

# Step 7: Define Hyperparameter Search Space
param_space = {
    'alpha': hp.uniform('alpha', 0.1, 0.5),  # Learning rate (0.1 - 0.5)
    'gamma': hp.uniform('gamma', 0.5, 0.9),  # Discount factor (0.5 - 0.9)
    'epsilon': hp.uniform('epsilon', 0.1, 0.3),  # Exploration rate (10%-30%)
    'epsilon_decay': hp.uniform('epsilon_decay', 0.98, 0.999),  # Slower decay (98%-99.9%)
    'training_iterations': hp.quniform('training_iterations', 2, 10, 1)  # Replay transitions (2-10 times)
}

# Step 8: Run Hyperparameter Optimization
trials = Trials()
best_params = fmin(fn=train_rl, space=param_space, algo=tpe.suggest, max_evals=50, trials=trials)

print(f"Best RL Parameters: {best_params}")


In [9]:
# # Convert Post-Depletion Data to GeoDataFrame for Visualization
# df_post_depletion = gpd.GeoDataFrame(df_post_depletion, 
#                                      geometry=gpd.points_from_xy(df_post_depletion['Log_Grid'], df_post_depletion['Lat_Grid']), 
#                                      crs="EPSG:4326")

# # Convert Predicted Grid Cells to Points
# predicted_points = df_post_depletion[['Predicted_Grid_Cell']].dropna().copy()
# predicted_points['Lat_Pred'] = predicted_points['Predicted_Grid_Cell'].apply(lambda x: grid_gdf.loc[x, 'geometry'].centroid.y if x in grid_gdf.index else None)
# predicted_points['Lon_Pred'] = predicted_points['Predicted_Grid_Cell'].apply(lambda x: grid_gdf.loc[x, 'geometry'].centroid.x if x in grid_gdf.index else None)
# predicted_points = predicted_points.dropna()

# # Convert to GeoDataFrame
# gdf_predicted = gpd.GeoDataFrame(predicted_points, 
#                                  geometry=gpd.points_from_xy(predicted_points['Lon_Pred'], predicted_points['Lat_Pred']), 
#                                  crs="EPSG:4326")

# # Plot the actual and predicted trajectories
# fig, ax = plt.subplots(figsize=(12, 12))

# # Plot Grid
# grid_gdf.plot(ax=ax, color='lightgrey', edgecolor='grey', alpha=0.4)

# # Plot Actual Trajectory (Post-Depletion)
# df_post_depletion.plot(ax=ax, color='red', markersize=20, label="Actual Trajectory", alpha=0.7)

# # Plot Predicted Trajectory
# gdf_predicted.plot(ax=ax, color='blue', markersize=20, label="Predicted Trajectory", alpha=0.7)

# # Add Basemap
# ctx.add_basemap(ax, crs=df_post_depletion.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)

# # Formatting
# ax.set_title("Actual vs Predicted Trajectory (Post-Depletion)", fontsize=14)
# ax.legend()
# ax.set_axis_off()

# plt.show()


# **Mathematical Formulation of Post-Depletion Trajectory Analysis**

This analysis enables a detailed comparison of **actual vs predicted movement** following battery depletion, revealing:
- How vehicles move after depletion.
- The effectiveness of the **Markov-based prediction model**.
- Patterns in vehicle trajectory shifts post-depletion.

## **1. Extracting Unique Days of Battery Depletion**
We define a unique day $ d $ as a calendar date where at least one vehicle experienced battery depletion:

$$
D = \{ d \mid \exists i, SOC_{\text{batt}, i} < 50\%, \text{on day } d \}
$$

where $ D $ is the set of all days in which a depletion event occurred.

## **2. Filtering Data for Each Depletion Day**
For each $ d \in D $, we extract:

- **Post-depletion records**: 
  $$
  X_d = \{ x_i \mid x_i \in X, \text{Timestamp}(x_i) = d, SOC_{\text{batt}, i} < 50\% \}
  $$

- **Pre-depletion records**:
  $$
  Y_d = \{ y_i \mid y_i \in Y, \text{Timestamp}(y_i) = d, SOC_{\text{batt}, i} \geq 50\% \}
  $$

where:
- $ X_d $ represents all vehicle records **after depletion**.
- $ Y_d $ represents all vehicle records **before depletion**.

## **3. Mapping Vehicles to Grid Cells**
For each vehicle's recorded GPS point $ p_i $ on day $ d $:

$$
G_i = \arg\max_j \mathbb{1}(p_i \in P_j)
$$

where:
- $ G_i $ is the assigned grid cell.
- $ P_j $ represents the grid cells.
- $ \mathbb{1}(p_i \in P_j) $ is an indicator function that is **1** if $ p_i $ is inside $ P_j $.

Thus, we define:

$$
G_d^{\text{actual}} = \{ G_i \mid x_i \in X_d \}
$$

$$
G_d^{\text{pre}} = \{ G_i \mid y_i \in Y_d \}
$$

## **4. Extracting Predicted Grid Cells**
The predicted post-depletion grid cells for each vehicle are computed from the **Markov Transition Model**:

$$
G_{\text{predicted}, i} = \arg\max_{G_j} P(G_j | G_i)
$$

for each vehicle location $ G_i $ before depletion.

The predicted set is:

$$
G_d^{\text{predicted}} = \{ G_{\text{predicted}, i} \mid x_i \in X_d \}
$$

## **5. Visualizing the Spatial Trajectories**
We generate a geospatial plot for each day $ d $:

- **Pre-Depletion Trajectory** $ G_d^{\text{pre}} $ (Black)
- **Actual Post-Depletion Trajectory** $ G_d^{\text{actual}} $ (Red)
- **Predicted Trajectory** $ G_d^{\text{predicted}} $ (Blue)

Each grid cell is represented as a polygon, where:

$$
P_j = \{ (x_k, y_k) \mid k = 1,2,3,4 \}
$$

and plotted based on its category:

$$
\text{Color}(P_j) =
\begin{cases} 
\text{black}, & P_j \in G_d^{\text{pre}} \\
\text{red}, & P_j \in G_d^{\text{actual}} \\
\text{blue}, & P_j \in G_d^{\text{predicted}}
\end{cases}
$$

## **6. Overlaying the OpenStreetMap Basemap**
The plotted grid cells are projected onto a real-world **OpenStreetMap** (OSM) basemap with coordinate reference system:

$$
\text{CRS} = \text{EPSG:4326}
$$

In [ ]:
# Extract unique days where depletion occurred
depleted_days = df_post_depletion['Timestamp'].dt.date.unique()

# Loop through each depleted day and generate a plot
for day in depleted_days:
    # Filter data for the current day
    df_day = df_post_depletion[df_post_depletion['Timestamp'].dt.date == day]
    df_pre_depletion_day = df_pre_depletion[df_pre_depletion['Timestamp'].dt.date == day]

    # Convert actual, predicted, and pre-depletion data into GeoDataFrames
    gdf_actual = grid_gdf.loc[grid_gdf.index.isin(df_day['Grid_Cell'])].copy()
    gdf_actual['Color'] = 'red'

    predicted_grid_cells = df_day['Predicted_Grid_Cell'].dropna().unique()
    gdf_predicted = grid_gdf.loc[grid_gdf.index.isin(predicted_grid_cells)].copy()
    gdf_predicted['Color'] = 'blue'

    pre_depletion_grid_cells = df_pre_depletion_day['Grid_Cell'].unique()
    gdf_pre_depletion = grid_gdf.loc[grid_gdf.index.isin(pre_depletion_grid_cells)].copy()
    gdf_pre_depletion['Color'] = 'black'

    # Skip plotting if all GeoDataFrames are empty for the day
    if gdf_actual.empty and gdf_predicted.empty and gdf_pre_depletion.empty:
        print(f"Skipping {day}: No valid data for plotting.")
        continue

    # Create the plot for the current day
    fig, ax = plt.subplots(figsize=(12, 12))

    # Plot Grid Cells
    grid_gdf.plot(ax=ax, color='lightgrey', edgecolor='grey', alpha=0.2)

    # Plot Pre-Depletion Trajectory (Black) if not empty
    if not gdf_pre_depletion.empty:
        gdf_pre_depletion.plot(ax=ax, color='black', alpha=0.5, edgecolor='black', label="Pre-Depletion Trajectory")

    # Plot Actual Trajectory (Red) if not empty
    if not gdf_actual.empty:
        gdf_actual.plot(ax=ax, color='red', alpha=0.5, edgecolor='red', label="Actual Trajectory")

    # Plot Predicted Trajectory (Blue) if not empty
    if not gdf_predicted.empty:
        gdf_predicted.plot(ax=ax, color='blue', alpha=0.5, edgecolor='blue', label="Predicted Trajectory")

    # Add Basemap
    try:
        ctx.add_basemap(ax, crs=grid_gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
    except Exception as e:
        print(f"Basemap Error on {day}: {e}")

    # Formatting
    ax.set_title(f"Actual vs Predicted Trajectory (Post-Depletion) - {day}", fontsize=14)
    ax.legend()
    ax.set_axis_off()

    # Show plot
    plt.show()


In [ ]:
df_temp = df.copy().reset_index(drop=True)  # Ensure indices are sequential
df_temp=df_temp.sort_values(by=['Timestamp']).reset_index(drop=True) 

# Define different time thresholds to compare
time_thresholds = {
    "3 sec": 3,
    "12 sec": 12
}

# Create a dictionary to store SOC and sensor states for each threshold
soc_depletion_results = {}

# Iterate over different time thresholds
for label, TIME_THRESHOLD in time_thresholds.items():

    # Track last sensed timestamp, and stored energy during OFF periods
    last_sensed_time = {}
    stored_energy = {}

    # Previous date
    prev_date=None
    
    for i in range(len(df_temp)):
        row = df_temp.iloc[i]
        grid_key = (row['Lat_Grid'], row['Log_Grid'])  # Unique grid identifier
        current_time = row['Timestamp']
        current_date = row['Date']
        device= row['deviceID']

        # Initialise inter-row differences when OFF
        d_diff_prev=0 
        
        # Reset stored energy at the start of a new day
        if prev_date is not None and current_date != prev_date:
            stored_energy={}  # Reset stored energy for all grid cells
            df_temp.loc[i:, f'Energy_Saved_{label}'] = 0  # Reset energy savings for the new day
            print(f"[RESET] Reset stored energy for new day: {current_date}")

        prev_date = current_date  # Update previous date tracker

        if df_temp.loc[i, 'SOC_batt']>99:
            stored_energy[grid_key]=0
            df_temp.loc[i:, f'Energy_Saved_{label}']=0

        # If the grid was sensed recently (within the threshold), turn OFF the sensor
        if grid_key in last_sensed_time and (current_time - last_sensed_time[grid_key]).total_seconds() < TIME_THRESHOLD:
            df_temp.at[i, f'Sensor_ON_{label}'] = False   

            # Accumulate stored energy
            if i > 0 and pd.notna(df_temp.iloc[i - 1]['SOC_batt']) and pd.notna(row['SOC_batt']):
            
                # Find the last preceding row for this device
                if device == df_temp.iloc[i-1]['deviceID']:
                    d_diff = max(0, df_temp.iloc[i - 1]['SOC_batt'] - row['SOC_batt']) #current inter-row difference
                    diff = max(d_diff_prev, d_diff)
                    stored_energy[grid_key] = df_temp.loc[i-1, f'Energy_Saved_{label}']
                    stored_energy[grid_key] += diff
                    
                    if diff != 0: 
                        df_temp.loc[i:, f'Energy_Saved_{label}'] = stored_energy[grid_key]
                    else:
                        df_temp.loc[i:, f'Energy_Saved_{label}'] = df_temp.loc[i-1, f'Energy_Saved_{label}']

                    d_diff_prev=diff
                    print(f"[OFF]: Accumulated {diff:.2f}% for device {device}. Total stored: {stored_energy[grid_key]:.2f}%")

                else:
                    d_diff = max(0, df_temp.loc[df_temp.deviceID == device, :]['SOC_batt'].iloc[-1] - row['SOC_batt']) #current inter-row difference
                    diff = max(d_diff_prev, d_diff)
                    stored_energy[grid_key] = df_temp.loc[df_temp.deviceID == device, :][f'Energy_Saved_{label}'].iloc[-1]
                    stored_energy[grid_key] += diff
                    
                    if diff != 0: 
                        df_temp.loc[i:, f'Energy_Saved_{label}'] = stored_energy[grid_key]
                    else:
                        df_temp.loc[i:, f'Energy_Saved_{label}'] = df_temp.loc[df_temp.deviceID == device, :][f'Energy_Saved_{label}'].iloc[-1]

                    d_diff_prev=diff
                    print(f"CHANGE [OFF]: Accumulated {diff:.2f}% for device {device}. Total stored: {stored_energy[grid_key]:.2f}%")


        else:
            # Update last sensed time when the sensor turns ON
            last_sensed_time[grid_key] = current_time
            df_temp.at[i, f'Sensor_ON_{label}'] = True
            d_diff_prev=0

            if device == df_temp.iloc[i-1]['deviceID']:

                # Ensure stored_energy is initialized per grid cell without overwriting previous values
                if grid_key not in stored_energy:
                    if i > 0:
                        #Carry forward the stored energy from the last known row
                        stored_energy[grid_key] = df_temp.loc[i-1, f'Energy_Saved_{label}']
                        df_temp.loc[i:, f'Energy_Saved_{label}'] = stored_energy[grid_key]
                    elif i == 0:
                        stored_energy[grid_key] = 0  # First iteration, no prior energy 
                        df_temp.loc[i:, f'Energy_Saved_{label}'] = stored_energy[grid_key]  
                print(f"[ON]: device {device}. Total stored: {stored_energy[grid_key]:.2f}%")


            else:

                 # Ensure stored_energy is initialized per grid cell without overwriting previous values
                if grid_key not in stored_energy:
                    if i > 0:
                        #Carry forward the stored energy from the last known row
                        stored_energy[grid_key] = df_temp.loc[df_temp.deviceID == device, :][f'Energy_Saved_{label}'].iloc[-1]
                        df_temp.loc[i:, f'Energy_Saved_{label}'] = stored_energy[grid_key]
                    elif i == 0:
                        stored_energy[grid_key] = 0  # First iteration, no prior energy 
                        df_temp.loc[i:, f'Energy_Saved_{label}'] = stored_energy[grid_key]                 
                
                print(f"CHANGE [ON]: device {device}. Total stored: {stored_energy[grid_key]:.2f}%")

    
    # Compute new SOC_batt with savings
    df_temp[f'SOC_batt_{label}'] = df_temp['SOC_batt'] + df_temp[f'Energy_Saved_{label}']
    df_temp[f'SOC_batt_{label}'] = df_temp[f'SOC_batt_{label}'].clip(upper=100)

    # Compute SOC depletion for this threshold
    daily_soc = df_temp.groupby(['Date', 'deviceID'])[f'SOC_batt_{label}'].mean()
    soc_depletion_results[label] = daily_soc


# Baseline: Compute SOC depletion without constraints
soc_depletion_results["Baseline"] = df_temp.groupby(['Date', 'deviceID'])['SOC_batt'].mean()

# Convert results to a DataFrame for plotting
soc_depletion_df = pd.DataFrame(soc_depletion_results)

# Save the updated dataset with sensor states and energy savings for each threshold
output_path = "/Users/mayar/Desktop/MIT/Research_Fellow/ENERGY_SENSING/DATA/updated_SOC_batt_with_energy_savings.xlsx"
df_temp.to_excel(output_path, index=False)



In [ ]:
# Define line styles for each threshold
line_styles = {
    "Baseline": "-",
    "3 sec": "--",
    "12 sec": ":"
}

# Predefined colors for devices
predefined_colors = ['red', 'blue', 'green', 'black', 'purple']
device_ids = set()
for soc_series in soc_depletion_results.values():
    device_ids.update(soc_series.index.get_level_values('deviceID').unique())

# Create a color map using predefined colors
color_map = {device_id: predefined_colors[i % len(predefined_colors)] for i, device_id in enumerate(sorted(device_ids))}

# Plot SOC depletion for different devices and thresholds
plt.figure(figsize=(12, 6))

# Iterate over thresholds and plot per device
for label, soc_series in soc_depletion_results.items():  # soc_series is a MultiIndexed Series
    for device_id in soc_series.index.get_level_values('deviceID').unique():  # Get unique devices
        device_data = soc_series[soc_series.index.get_level_values('deviceID') == device_id]
        plt.plot(
            device_data.index.get_level_values('Date'),  # X-axis: Dates
            device_data.values,  # Y-axis: SOC values
            linestyle=line_styles[label],
            color=color_map[device_id],  # Use predefined color for the device
            marker='o',
            markersize=3,
            label=f"Device {device_id} - {label}"
        )

plt.xlabel('Date')
plt.ylabel('Mean SOC (%)')
plt.title('SOC Depletion Comparison Across Devices and Time Constraints')

# Place the legend outside the plot
plt.legend(
    bbox_to_anchor=(1.05, 1),  # Place legend to the right of the plot
    loc='upper left',          # Align legend to the top-left of the bounding box
    borderaxespad=0.           # Reduce spacing between the legend and the plot
)
plt.grid(True)
plt.xticks(rotation=45)

# Adjust layout to make room for the legend
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
# Compute the count of times the sensor was turned OFF for each constraint scenario
off_counts = {}

# Iterate over different time thresholds
for label, TIME_THRESHOLD in time_thresholds.items():
    df_copy = df.copy()  # Work on a copy of the dataset
    df_copy['Sensor_ON'] = True  # Default: Sensor is ON

    # Track last sensed timestamp per grid cell
    last_sensed_time = {}
    off_count = 0

    for i, row in df_copy.iterrows():
        grid_key = (row['Lat_Grid'], row['Log_Grid'])  # Unique grid identifier
        current_time = row['Timestamp']

        # If the grid was sensed recently (within the threshold), turn OFF the sensor
        if grid_key in last_sensed_time and (current_time - last_sensed_time[grid_key]).total_seconds() < TIME_THRESHOLD:
            df_copy.at[i, 'Sensor_ON'] = False
            off_count += 1
        else:
            # Update last sensed time when the sensor turns ON
            last_sensed_time[grid_key] = current_time

    off_counts[label] = off_count

# Convert to DataFrame for better visualization
off_counts_df = pd.DataFrame.from_dict(off_counts, orient='index', columns=['Sensor OFF Count'])

# Display results
off_counts_df

In [ ]:
# Extract unique days where depletion occurred
depleted_days = df_post_depletion['Timestamp'].dt.date.unique()

# Loop through each depleted day for visualization
for day in depleted_days:
    # Filter data for the current depleted day
    df_day_pre = df_pre_coverage[df_pre_coverage['Date'] == day]
    df_day_new = df_new_coverage_only[df_new_coverage_only['Date'] == day]
    df_day_actual = df_post_depletion[df_post_depletion['Timestamp'].dt.date == day]
    df_day_predicted = df_post_depletion[df_post_depletion['Timestamp'].dt.date == day]

    # Convert to GeoDataFrames
    gdf_pre = grid_gdf.loc[grid_gdf.index.isin(df_day_pre['Grid_Cell'])].copy()
    gdf_pre['Color'] = 'black'  # Pre-depletion trajectory

    gdf_new = grid_gdf.loc[grid_gdf.index.isin(df_day_new['Grid_Cell'])].copy()
    gdf_new['Color'] = 'green'  # Newly sensed due to 10min rule

    gdf_actual = grid_gdf.loc[grid_gdf.index.isin(df_day_actual['Grid_Cell'])].copy()
    gdf_actual['Color'] = 'red'  # Actual trajectory after depletion

    predicted_grid_cells = df_day_predicted['Predicted_Grid_Cell'].dropna().unique()
    gdf_predicted = grid_gdf.loc[grid_gdf.index.isin(predicted_grid_cells)].copy()
    gdf_predicted['Color'] = 'blue'  # Predicted trajectory

    # Skip if no relevant data for the day
    if gdf_pre.empty and gdf_new.empty and gdf_actual.empty and gdf_predicted.empty:
        print(f"Skipping {day}: No valid data for plotting.")
        continue

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 12))

    # Plot Grid Cells (Background)
    grid_gdf.plot(ax=ax, color='lightgrey', edgecolor='grey', alpha=0.2)

    # Plot Pre-Depletion Trajectory (Black)
    if not gdf_pre.empty:
        gdf_pre.plot(ax=ax, color='black', alpha=0.5, edgecolor='black', label="Pre-Depletion Trajectory")

    # Plot Actual Post-Depletion Trajectory (Red)
    if not gdf_actual.empty:
        gdf_actual.plot(ax=ax, color='red', alpha=0.5, edgecolor='red', label="Actual Trajectory (Post-Depletion)")

    # Plot Predicted Post-Depletion Trajectory (Blue)
    if not gdf_predicted.empty:
        gdf_predicted.plot(ax=ax, color='blue', alpha=0.5, edgecolor='blue', label="Predicted Trajectory")

    # Plot Newly Sensed Cells Due to 10-Minute Rule (Green)
    if not gdf_new.empty:
        gdf_new.plot(ax=ax, color='green', alpha=0.5, edgecolor='green', label="Newly Sensed Cells (10-min Interval)")

    # Add Basemap
    try:
        ctx.add_basemap(ax, crs=grid_gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
    except Exception as e:
        print(f"Basemap Error on {day}: {e}")

    # Formatting
    ax.set_title(f"Trajectory Visualization with 10-Minute Sensing Constraint - {day}", fontsize=14)
    ax.legend()
    ax.set_axis_off()

    # Show plot
    plt.show()


In [ ]:
# Dynamically get the bounds from the data
min_lat, max_lat = df['Lat'].min(), df['Lat'].max()
min_lon, max_lon = df['Log'].min(), df['Log'].max()

# Define grid size (120x120 meters)
grid_size = 120
lat_resolution = grid_size / 111320  # Convert meters to latitude degrees
lon_resolution_at_lat = lambda lat: grid_size / (111320 * np.cos(np.radians(lat)))

# Generate grid covering the dataset area
grid = []
lat = min_lat
while lat < max_lat:
    lon = min_lon
    while lon < max_lon:
        lon_res = lon_resolution_at_lat(lat)
        grid.append(Polygon([
            (lon, lat),
            (lon + lon_res, lat),
            (lon + lon_res, lat + lat_resolution),
            (lon, lat + lat_resolution)
        ]))
        lon += lon_res
    lat += lat_resolution

# Create an empty GeoDataFrame for the grid
grid_gdf = gpd.GeoDataFrame({'geometry': grid, 'Count': 0}, crs="EPSG:4326")

# Create a GeoDataFrame for the data points
df_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Log'], df['Lat']), crs="EPSG:4326")

# Assign each measurement to a grid square
for index, point in df_gdf.iterrows():
    match = grid_gdf.contains(point.geometry)
    if match.any():
        grid_gdf.loc[match.idxmax(), 'Count'] += 1

# Apply Fractional Power Scaling
gamma = 0.3  # Adjust for visibility
grid_gdf['Scaled_Count'] = (grid_gdf['Count'] + 1) ** gamma

# Normalize values for color mapping
norm = Normalize(vmin=grid_gdf['Scaled_Count'].min(), vmax=grid_gdf['Scaled_Count'].max())
cmap = plt.get_cmap('jet')

# Convert scaled values to hex colors
grid_gdf['Color'] = grid_gdf['Scaled_Count'].apply(lambda x: to_hex(cmap(norm(x))))

# Create Folium map centered on Stockholm
m = folium.Map(location=[df['Lat'].mean(), df['Log'].mean()], zoom_start=12, tiles='Cartodb dark_matter')

# Function to color the grid based on scaled counts
def style_function(feature):
    color = feature['properties']['Color']  # Get precomputed color
    return {
        'fillColor': color,
        'color': 'black',
        'weight': 0.1,
        'fillOpacity': 0.4
    }

# Add grid layer to Folium
folium.GeoJson(
    grid_gdf,
    name="Measurement Grid",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['Count'], aliases=["Measurements:"])
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Display the map
m


### Initial Analysis
#### Check Data Distribution
- Before plotting, inspect the distribution of the Count column to confirm the skew. If the Count values have a large range (e.g., some counts are much higher than others), you can apply a logarithmic scale to the color mapping. This makes smaller variations more distinguishable.


In [ ]:
# Group by spatial grid and count occurrences
coverage = df.groupby(['Lat_Grid', 'Log_Grid']).size().reset_index(name='Count')

# Count the frequency of each unique coverage count
coverage_freq = coverage['Count'].value_counts().reset_index()
coverage_freq.columns = ['Coverage Count', 'Frequency']

# Sort in descending order
coverage_freq = coverage_freq.sort_values(by='Coverage Count', ascending=False)

# Find the maximum coverage count
max_count = coverage['Count'].max()

sns.histplot(coverage['Count'], bins=max_count, kde=True, color='blue')
plt.title('Distribution of Coverage Counts')
plt.xlabel('Coverage Count')
plt.ylabel('Frequency')
plt.ylim(0, 2)
plt.show()


In [113]:
# df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')

# # Define Zipf-Mandelbrot function
# def zipf_mandelbrot_func(r, s, q, C):
#     return C / (r + np.abs(q)) ** s  # Ensure q is positive

# # Define resolutions to test
# spatial_resolutions = [0.00001, 0.0001, 0.001, 0.01]
# temporal_resolutions = ['10S', '30S', '1T', '5T']

# # Store results
# results = []

# for spatial_resolution in spatial_resolutions:
#     for temporal_resolution in temporal_resolutions:
#         # Create spatial grid
#         df['Lat_Grid'] = (df['Lat'] // spatial_resolution) * spatial_resolution
#         df['Log_Grid'] = (df['Log'] // spatial_resolution) * spatial_resolution
        
#         # Create temporal bins
#         df['Time_Bin'] = df['Timestamp'].dt.floor(temporal_resolution)

#         # Group by spatial grid and count occurrences
#         coverage = df.groupby(['Lat_Grid', 'Log_Grid']).size().reset_index(name='Count')

#         # Sort data in Zipfian order
#         sorted_counts = np.sort(coverage['Count'])[::-1]  # Descending order
#         ranks = np.arange(1, len(sorted_counts) + 1)  # Rank numbers

#         # Fit Zipf-Mandelbrot
#         try:
#             params, _ = curve_fit(zipf_mandelbrot_func, ranks, sorted_counts, 
#                                   p0=[1, 1, max(sorted_counts)], 
#                                   bounds=([0.5, 0.0001, 0], [3, 10, np.inf]))

#             s_fit, q_fit, C_fit = params
#             expected_values = zipf_mandelbrot_func(ranks, s_fit, q_fit, C_fit)
            
#             # Compute residuals
#             residuals = sorted_counts - expected_values
#             std_residuals = np.std(residuals)
            
#             # Perform KS test
#             ks_stat, p_value = kstest(sorted_counts, zipf_mandelbrot_func, args=(s_fit, q_fit, C_fit))

#             # Compute AIC (Akaike Information Criterion)
#             AIC = -2 * np.log(p_value) + 2 * 3  # 3 parameters: s, q, C

#             # Store results
#             results.append({
#                 'Spatial_Resolution': spatial_resolution,
#                 'Temporal_Resolution': temporal_resolution,
#                 'KS_Statistic': ks_stat,
#                 'p_value': p_value,
#                 'Std_Residuals': std_residuals,
#                 'AIC': AIC
#             })

#         except RuntimeError:
#             print(f"Fit failed for Spatial={spatial_resolution}, Temporal={temporal_resolution}")

# # Convert results to DataFrame
# results_df = pd.DataFrame(results)

# # Select the best resolution (min AIC, high p-value, low KS statistic)
# best_result = results_df.sort_values(by=['AIC', 'KS_Statistic'], ascending=[True, True]).iloc[0]
# print("Best Resolution Parameters:")
# print(best_result)


In [ ]:
# Sort coverage counts in descending order (ranked frequencies)
sorted_counts = np.sort(coverage['Count'])[::-1]  # Descending order
ranks = np.arange(1, len(sorted_counts) + 1)  # Rank numbers

# Define Zipf-Mandelbrot function: f(r) = C / (r + q)^s
def zipf_mandelbrot_func(r, s, q, C):
    return C / (r + np.abs(q)) ** s  # Ensure q is positive

# Fit Zipf-Mandelbrot with constraints to avoid numerical issues
params, _ = curve_fit(zipf_mandelbrot_func, ranks, sorted_counts, p0=[1, 1, max(sorted_counts)], bounds=([0.5, 0.0001, 0], [3, 10, np.inf]))
s_fit, q_fit, C_fit = params

# Compute expected values from the fitted Zipf-Mandelbrot model
expected_values = zipf_mandelbrot_func(ranks, s_fit, q_fit, C_fit)

# Compute residuals (Observed - Expected)
residuals = sorted_counts - expected_values
relative_residuals = residuals / expected_values  # Normalize residuals

# Plot Residuals
plt.figure(figsize=(10, 6))
plt.scatter(ranks, residuals, alpha=0.6, color="red", label="Residuals (Observed - Expected)")
plt.axhline(0, linestyle="--", color="black", alpha=0.6)
plt.xscale("log")
plt.yscale("linear")
plt.xlabel("Rank", fontsize=12)
plt.ylabel("Residual (Observed - Expected)", fontsize=12)
plt.title("Residuals from Zipf-Mandelbrot Fit", fontsize=14)
plt.legend()
plt.grid(True, which="both", linestyle="--", alpha=0.5)
plt.show()


In [ ]:
# Define a threshold for outliers (e.g., 1.2x expected value)
tolerance_factor = 3

# Identify outliers (values too far from expected Zipfian behavior)
outlier_mask = (sorted_counts > expected_values * tolerance_factor) | (sorted_counts < expected_values / tolerance_factor)

# Count the number of removed points
num_outliers = outlier_mask.sum()
print(f"Number of detected outliers: {num_outliers}")

# Remove outliers from dataset
filtered_counts = sorted_counts[~outlier_mask]
filtered_ranks = ranks[~outlier_mask]

# Exclude extreme values (top 5% and bottom 5%)
lower_bound_c = int(0.1 * len(filtered_counts))
upper_bound_c = int(0.9 * len(filtered_counts))
filtered_counts=filtered_counts[lower_bound_c:upper_bound_c]
lower_bound_r = int(0.1 * len(filtered_ranks))
upper_bound_r = int(0.9 * len(filtered_ranks))
filtered_ranks = filtered_ranks[lower_bound_r:upper_bound_r]

# Re-Fit Zipf-Mandelbrot with filtered data
params_filtered, _ = curve_fit(
    zipf_mandelbrot_func, 
    filtered_ranks, 
    filtered_counts, 
    p0=[1, 1, max(filtered_counts)], 
    bounds=([0.5, 0.0001, 0], [3, 10, np.inf])
)

# Extract new parameters
s_fit_filtered, q_fit_filtered, C_fit_filtered = params_filtered

# Compute expected values with new parameters
expected_values_filtered = zipf_mandelbrot_func(filtered_ranks, s_fit_filtered, q_fit_filtered, C_fit_filtered)

# Plot cleaned data vs. new Zipf-Mandelbrot fit
plt.figure(figsize=(10, 6))
plt.scatter(filtered_ranks, filtered_counts, label="Filtered Data (No Outliers)", alpha=0.6, color="blue")
plt.plot(filtered_ranks, expected_values_filtered, 'r-', linewidth=2, label="Re-Fitted Zipf-Mandelbrot Model")

plt.xscale("log")
plt.yscale("log")
plt.xlabel("Rank", fontsize=12)
plt.ylabel("Coverage Count", fontsize=12)
plt.title("Zipf-Mandelbrot Fit After Outlier Removal & Re-Fitting", fontsize=14)
plt.legend()
plt.grid(True, which="both", linestyle="--", alpha=0.5)

plt.show()


In [ ]:
# Exclude extreme values (top 5% and bottom 5%)
lower_bound = int(0.1 * len(filtered_counts))
upper_bound = int(0.9 * len(filtered_counts))

ks_stat_truncated, p_value_truncated = kstest(
    filtered_counts[lower_bound:upper_bound], 
    zipf_mandelbrot_func, 
    args=(s_fit_filtered, q_fit_filtered, C_fit_filtered)
)
ks_stat_after, p_value_after = kstest(filtered_counts, zipf_mandelbrot_func, args=(s_fit_filtered, q_fit_filtered, C_fit_filtered))


print(f"Truncated KS Test - Statistic: {ks_stat_truncated:.4f}, p-value: {p_value_truncated:.4f}")


In [ ]:
print(f"Estimated Zipf-Mandelbrot Exponent (s): {s_fit:.4f}")
from scipy.stats import norm

# Compute log-likelihood
log_likelihood = np.sum(norm.logpdf(filtered_counts, loc=zipf_mandelbrot_func(filtered_ranks, s_fit_filtered, q_fit_filtered, C_fit_filtered), scale=np.std(filtered_counts)))
AIC_fixed = -2 * log_likelihood + 2 * 3  # 3 parameters: s, q, C

print(f"Fixed AIC: {AIC_fixed:.4f}")



In [ ]:
# Define pure Zipf function: f(r) = C / r^s
def zipf_func(r, s, C):
    return C / r ** s

# Fit pure Zipf
params_zipf, _ = curve_fit(zipf_func, filtered_ranks, filtered_counts, 
                           p0=[1, max(filtered_counts)], 
                           bounds=([0.5, 0], [3, np.inf]))

s_zipf, C_zipf = params_zipf
expected_values_zipf = zipf_func(filtered_ranks, s_zipf, C_zipf)

# Compute Log-Likelihood and AIC for pure Zipf
log_likelihood_zipf = np.sum(norm.logpdf(
    filtered_counts, 
    loc=expected_values_zipf, 
    scale=np.std(filtered_counts)
))
AIC_zipf = -2 * log_likelihood_zipf + 2 * 2  # 2 parameters: s, C

print(f"Pure Zipf Log-Likelihood: {log_likelihood_zipf:.4f}")
print(f"Pure Zipf AIC: {AIC_zipf:.4f}")


In [ ]:
# Top 10% (frequent ranks)
top_residuals = filtered_counts[:int(0.1 * len(filtered_counts))] - expected_values_filtered[:int(0.1 * len(filtered_counts))]

# Bottom 10% (rare ranks)
bottom_residuals = filtered_counts[-int(0.1 * len(filtered_counts)):] - expected_values_filtered[-int(0.1 * len(filtered_counts)):]

# Plot residuals
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(filtered_ranks[:int(0.1 * len(filtered_ranks))], top_residuals, color='red', alpha=0.6)
plt.axhline(0, linestyle='--', color='black', alpha=0.6)
plt.xscale('log')
plt.title("Top 10% Residuals")
plt.xlabel("Rank")
plt.ylabel("Residual")

plt.subplot(1, 2, 2)
plt.scatter(filtered_ranks[-int(0.1 * len(filtered_ranks)):], bottom_residuals, color='blue', alpha=0.6)
plt.axhline(0, linestyle='--', color='black', alpha=0.6)
plt.xscale('log')
plt.title("Bottom 10% Residuals")
plt.xlabel("Rank")
plt.ylabel("Residual")

plt.tight_layout()
plt.show()


In [ ]:
# Focus on middle 80% of ranks
lower_bound = int(0.1 * len(filtered_counts))
upper_bound = int(0.9 * len(filtered_counts))

ks_stat_truncated, p_value_truncated = kstest(
    filtered_counts[lower_bound:upper_bound], 
    zipf_mandelbrot_func, 
    args=(s_fit_filtered, q_fit_filtered, C_fit_filtered)
)

print(f"Truncated KS Test - Statistic: {ks_stat_truncated:.4f}, p-value: {p_value_truncated:.4f}")


In [ ]:
expected_values_filtered = zipf_mandelbrot_func(filtered_ranks, s_fit_filtered, q_fit_filtered, C_fit_filtered)
residuals_filtered = filtered_counts - expected_values_filtered


plt.figure(figsize=(10, 6))
plt.scatter(filtered_ranks, residuals_filtered, alpha=0.6, color="red", label="Residuals (Observed - Expected)")

plt.axhline(0, linestyle="--", color="black", alpha=0.6)
plt.xscale("log")
plt.yscale("linear")
plt.xlabel("Rank", fontsize=12)
plt.ylabel("Residual (Observed - Expected)", fontsize=12)
plt.title("Residuals After Outlier Removal", fontsize=14)
plt.legend()
plt.grid(True, which="both", linestyle="--", alpha=0.5)

plt.show()


### Plot
- Analyze sensor coverage by aggregating the spatial grid.
- Visualize coverage heatmaps. To better visualize the data, apply logarithmic scaling to the color values. This will compress the range of large values and expand the smaller values for more differentiation in color. 
- We apply Fractional Power Scaling: Highlights smaller values significantly, making subtle differences more visible. We Raise the log-transformed values to a fractional power $ \log(x+1)^{0.5} $. This amplifies small differences while keeping the general scale.
##### Note:
- If \( x = 0 \), the standard `np.log(x)` would result in an error because the logarithm of 0 is undefined. 
`np.log1p(x)` handles this safely by adding \( 1 \) to the input before computing the logarithm, ensuring it works for non-negative numbers, including \( 0 \).
- The square root further compresses the range of the values.  
It emphasizes smaller differences by reducing the impact of large values. For example:  
$\log(x+1)^{0.5} $ grows slower than $\log(x+1)$ as $\ x $ increases.

**This transformation is particularly useful for skewed data, for `Count` values, where:**

- Most data points are small.
- A few extreme values (outliers) dominate.




In [ ]:
# Set up the figure size
plt.figure(figsize=(10, 8))

# Apply logarithmic scaling
log_scaled_values = np.log1p(coverage['Count'])**0.5 

# Apply logarithmic scaling to color values
sc = plt.scatter(
    coverage['Log_Grid'], 
    coverage['Lat_Grid'], 
    c=log_scaled_values,  
    cmap='jet', 
    s=30, 
    edgecolor='k', 
    alpha=0.8
)

# Add a color bar with the original scale in the label
cbar = plt.colorbar(sc)
cbar.set_label('√(Log(Coverage Count + 1))', fontsize=12)
cbar.ax.tick_params(labelsize=10)

# Add labels and title with improved font sizes
plt.xlabel('Longitude Grid', fontsize=14)
plt.ylabel('Latitude Grid', fontsize=14)
plt.title('Coverage Heatmap with Logarithmic Scale', fontsize=16)

# Add grid lines for reference
plt.grid(visible=True, linestyle='--', alpha=0.6)

# Improve tick sizes for better readability
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()


### Analyse the Temporal Sampling

In [ ]:
# Calculate time difference in seconds between consecutive rows
df['Delta_t'] = df['Timestamp'].diff().dt.total_seconds()  

# Define an expected interval in seconds (e.g., 60 seconds)
expected_interval = 60

# Count the total number of occurrences of measurements
tot_count = df['Delta_t'].count()
print(f"Number of values: {tot_count}")

# Count the number of occurrences of low frequency measurements
highf_count = (df['Delta_t'] > expected_interval).sum()
print(f"Number of values higher than 60sec: {highf_count}")

# Count the number of occurrences of 0.0
zero_count = (df['Timestamp'].diff().dt.total_seconds() == 0.0).sum()
print(f"Number of values equal to 0.0sec: {zero_count}")


### Energy and Coverage Model Preparation
- Create columns to represent:

    - Whether a street segment is already covered.
    - Battery state changes.

##### Filtering out Outliners

In [220]:
# Calculate time difference in seconds between consecutive rows
df['Delta_t'] = df['Timestamp'].diff().dt.total_seconds()  

# Define a threshold for acceptable intervals (e.g., 60 seconds)
acceptable_threshold = 60   # in seconds

# Filter out rows with large Delta_t
df = df[df['Delta_t'] <= acceptable_threshold]

# Drop rows with Delta_t equal to zero
df = df[df['Delta_t'] > 0]

# Reset the index
df = df.reset_index(drop=True)

The battery capacity is 10,000 mAh, so SOC_Change (calculated from current and time) must be converted into a percentage of the total capacity before being added to `SOC_batt`.

### SOC Update Formula:
$SOC_{new} = SOC_{old} + \frac{\Delta SOC_{mAh}}{C_{batt}} \times 100$ 

and 

$\Delta SOC_{mAh} = -1 \times I_{batt} \times \Delta t$ (mAh change based on current and time)

Where:
- $SOC_{old}$ is $SOC_{batt}$
- $\ I_{batt} $: Net current (`current_batt`) in mA (positive for consumption, negative for storage).
- $\ \Delta t $: Time difference in hours between consecutive rows.
- $\ C_{batt} $: Battery capacity in mAh (10,000 mAh).


In [ ]:
# Battery capacity in mAh
battery_capacity = 10000

# Calculate time difference in hours between consecutive rows
df['Delta_t'] = df['Delta_t'] / 3600  # Time difference in hours

# Calculate SOC change (in %) using the corrected formula
df['SOC_Change'] = (-1 * df['current_batt'] * df['Delta_t'] / battery_capacity) * 100

# Set SOC_Change to 0 when SOC is saturated
df.loc[df['SOC_batt'] >= 90, 'SOC_Change'] = 0

# Ensure SOC values are capped between 0 and 100
df['SOC_batt'] = df['SOC_batt'] + df['SOC_Change']
df['SOC_batt'] = df['SOC_batt'].clip(lower=0, upper=100)

# Assume that is >90%, charging is stopped
df.loc[df['SOC_batt'] >= 90, 'SOC_Change'] = 0

# Preview the updated DataFrame
print(df[['Timestamp', 'Lat', 'Log', 'SOC_batt', 'SOC_Change', 'Delta_t', 'current_batt']])

In [ ]:
# Plot SOC over time
plt.figure(figsize=(12, 6))
plt.plot(df['Timestamp'], df['SOC_batt'], label='State of Charge (SOC)', color='blue', linewidth=1.5)
plt.xlabel('Time', fontsize=14)
plt.ylabel('SOC (%)', fontsize=14)
plt.title('Battery State of Charge Over Time', fontsize=16)
plt.grid(alpha=0.5, linestyle='--')
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
# Filter rows where SOC is increasing or decreasing
increasing = df[df['SOC_Change'] > 0]
decreasing = df[df['SOC_Change'] < 0]

print(f"Number of times SOC increases: {len(increasing)}")
print(f"Number of times SOC decreases: {len(decreasing)}")


In [ ]:
# Mark covered segments (spatio-temporal condition)
df['Is_Covered'] = df.duplicated(subset=['Lat_Grid', 'Log_Grid', 'Time_Bin'], keep='first')

# Preview the updated DataFrame
print(df.head())

In [ ]:
def energy_aware_switch(row):
    if row['Is_Covered'] and row['SOC_batt'] > 20:
        return 'OFF'
    elif not row['Is_Covered'] and row['SOC_batt'] > 10:
        return 'ON'
    else:
        return 'IDLE'

df['Sensor_State'] = df.apply(energy_aware_switch, axis=1)

df['Cumulative_Spatial_Coverage'] = (~df['Is_Covered']).cumsum()
temporal_coverage = df.groupby(['Lat_Grid', 'Log_Grid', 'Time_Bin']).size().reset_index(name='Frequency')

high_coverage_cells = temporal_coverage[temporal_coverage['Frequency'] > 2]
high_coverage_cells


In [ ]:
# Initialize cumulative coverage and energy tracking
df['Cumulative_Coverage'] = 0
df['Average_SOC'] = 0

# Initialize variables
cumulative_coverage = set()  # To store unique covered grid cells
soc_list = []  # To store SOC levels

# Simulate over the data
for idx, row in df.iterrows():
    # Update cumulative coverage if sensor is ON
    if row['Sensor_State'] == 'ON':
        cumulative_coverage.add((row['Lat_Grid'], row['Log_Grid']))

    # Update SOC tracking
    soc_list.append(row['SOC_batt'])

    # Update DataFrame
    df.at[idx, 'Cumulative_Coverage'] = len(cumulative_coverage)
    df.at[idx, 'Average_SOC'] = sum(soc_list) / len(soc_list)

# Preview results
print(df[['Timestamp', 'Cumulative_Coverage', 'Average_SOC']])


In [ ]:
# Plot cumulative coverage over time
plt.figure(figsize=(10, 6))
plt.plot(df['Timestamp'], df['Cumulative_Coverage'], label='Cumulative Coverage', color='b')
plt.xlabel('Time', fontsize=14)
plt.ylabel('Cumulative Coverage', fontsize=14)
plt.title('Cumulative Coverage Over Time', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

# Plot average SOC over time
plt.figure(figsize=(10, 6))
plt.plot(df['Timestamp'], df['Average_SOC'], label='Average SOC', color='g')
plt.xlabel('Time', fontsize=14)
plt.ylabel('Average SOC (%)', fontsize=14)
plt.title('Average SOC Over Time', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
# Baseline: All sensors ON
df['Baseline_Coverage'] = 0
df['Baseline_SOC'] = 0.0

# Initialize variables
baseline_coverage = set()
baseline_soc_list = []

for idx, row in df.iterrows():
    # Assume sensors are always ON
    baseline_coverage.add((row['Lat_Grid'], row['Log_Grid']))
    baseline_soc_list.append(row['SOC_batt'])

    # Update DataFrame
    df.at[idx, 'Baseline_Coverage'] = len(baseline_coverage)
    df.at[idx, 'Baseline_SOC'] = sum(baseline_soc_list) / len(baseline_soc_list)

# Compare cumulative coverage and SOC
print(df[['Timestamp', 'Cumulative_Coverage', 'Baseline_Coverage', 'Average_SOC', 'Baseline_SOC']])


In [ ]:
# Total cumulative coverage
energy_aware_coverage = df['Cumulative_Coverage'].iloc[-1]
baseline_coverage = df['Baseline_Coverage'].iloc[-1]

# Average SOC
energy_aware_avg_soc = df['Average_SOC'].mean()
baseline_avg_soc = df['Baseline_SOC'].mean()

# Improvement metrics
coverage_improvement = (energy_aware_coverage - baseline_coverage) / baseline_coverage * 100
soc_savings = (baseline_avg_soc - energy_aware_avg_soc) / baseline_avg_soc * 100

# Print results
print(f"Energy-Aware Total Coverage: {energy_aware_coverage}")
print(f"Baseline Total Coverage: {baseline_coverage}")
print(f"Coverage Improvement: {coverage_improvement:.2f}%")
print(f"Energy Savings: {soc_savings:.2f}%")
